In [1]:
import numpy as np
import pandas as pd
from torch_geometric.data import HeteroData
import torch
import torch_geometric.transforms as T
import pickle
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, to_hetero

In [2]:

# Usage
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"



In [3]:

class HeteroDataProcessorFilterNodeonTest:
    def __init__(self, file_path_replies, file_path_posts, time_cut=15):
        self.file_path_replies = file_path_replies
        self.file_path_posts = file_path_posts
        self.time_cut = time_cut
        self.df_replies = None
        self.df_posts = None

    def load_data(self):
        self.df_replies = pd.read_pickle(self.file_path_replies)
        self.df_posts = pd.read_pickle(self.file_path_posts)

    def process_data(self):
        # Define post and reply features
        post_features = ['followers', 'favorite_count', 'retweet_count', 'verified', 'rumour', 'id', 'embeddings_avg']
        reply_features = ['reply_followers', 'reply_user_id', 'reply_verified', 'time_diff', 'reply_embeddings_avg', 'id']

        # Filter and group replies
        self.df_replies['min_since_fst_post'] = round(
            (self.df_replies['time'] - self.df_replies['time'].min()).dt.total_seconds() / 60, 2)
        grouped_replies = self.df_replies.groupby(['id']).agg(
            replies=('time_diff', 'count'),
            first_time_diff=('time_diff', 'first')
        ).reset_index()

        # Merge posts and replies
        self.df_posts = self.df_posts[post_features].merge(grouped_replies, on="id", how="inner")
        self.df_posts['replies'] = self.df_posts['replies'].fillna(0)
        self.df_posts['first_time_diff'] = self.df_posts['first_time_diff'].fillna(0)

        # One-hot encoding for verified columns
        self.df_posts['verified'] = self.df_posts['verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        self.df_posts = pd.concat([self.df_posts, pd.get_dummies(self.df_posts["verified"], dtype=int)], axis=1)
        self.df_posts.drop(["verified"], axis=1, inplace=True)
        self.df_posts.rename(columns={1: 'verified', 0: 'no_verified'}, inplace=True)

        self.df_replies['reply_verified'] = self.df_replies['reply_verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        self.df_replies = pd.concat([self.df_replies, pd.get_dummies(self.df_replies["reply_verified"], dtype=int)], axis=1)
        self.df_replies.drop(["reply_verified"], axis=1, inplace=True)
        self.df_replies.rename(columns={1: 'reply_verified', 0: 'reply_no_verified'}, inplace=True)

        # Train/test split
        train, not_train = train_test_split(self.df_posts, test_size=0.3, random_state=42, stratify=self.df_posts['rumour'])
        val, test = train_test_split(not_train, test_size=0.5, random_state=42, stratify=not_train['rumour'])

        # Post features processing
        post_features = train[["followers", "favorite_count", "retweet_count", "no_verified", "verified", "first_time_diff"]]
        scaler_posts = RobustScaler()
        scaled_features =scaler_posts.fit_transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])
        scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])
        scaled_data['no_verified'] = np.array(train['no_verified'])
        scaled_data['verified'] = np.array(train['verified'])
        post_features = scaled_data
        post_embeddings = np.array(train['embeddings_avg'].tolist())
        x1 = np.concatenate((post_features, post_embeddings), axis=1)

        # Reply features processing
        scaler_replies = RobustScaler()
        reply_features = self.df_replies[self.df_replies.id.isin(np.array(train.id))][["reply_followers", "reply_no_verified", "reply_verified", "time_diff"]]
        reply_features[['reply_followers', 'time_diff']] = scaler_replies.fit_transform(reply_features[['reply_followers', 'time_diff']])
        reply_embeddings = np.array(self.df_replies[self.df_replies.id.isin(np.array(train.id))]['reply_embeddings_avg'].tolist())
        x2 = np.concatenate((reply_features, reply_embeddings), axis=1)

        # Test/validation data preparation
        test_val_df_replies = pd.read_pickle(self.file_path_replies)
        test_val_df_posts = pd.read_pickle(self.file_path_posts)
        test_val_df_posts = test_val_df_posts[~test_val_df_posts.id.isin(train.id)]
        test_val_df_replies = test_val_df_replies[~test_val_df_replies.id.isin(train.id)]

        post_features = ['followers', 'favorite_count', 'retweet_count', 'verified', 'rumour', 'id', 'embeddings_avg']
        test_val_reply_features = ['reply_followers', 'reply_user_id', 'reply_verified', 'time_diff', 'reply_embeddings_avg', 'id']
        
        test_val_df_replies['min_since_fst_post'] = round((test_val_df_replies['time'] - test_val_df_replies['time'].min())\
        .dt.total_seconds() / 60,2)
        
        test_val_df_replies = test_val_df_replies[test_val_reply_features][(test_val_df_replies.time_diff <= time_cut)&\
           (test_val_df_replies.min_since_fst_post <= time_cut)]
        grouped_replies = test_val_df_replies.groupby(['id']).agg(
            replies=('time_diff', 'count'),
            first_time_diff=('time_diff', 'first')
        ).reset_index()
        
        test_val_df_posts = test_val_df_posts[post_features].merge(grouped_replies, on="id", how="inner")
        test_val_df_posts['replies'] = test_val_df_posts['replies'].fillna(0)
        test_val_df_posts['first_time_diff'] = test_val_df_posts['first_time_diff'].fillna(0)
        
        # One-hot encoding for verified columns
        test_val_df_posts['verified'] = test_val_df_posts['verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        test_val_df_posts = pd.concat([test_val_df_posts, pd.get_dummies(test_val_df_posts["verified"], dtype=int)], axis=1)
        test_val_df_posts.drop(["verified"], axis=1, inplace=True)
        test_val_df_posts.rename(columns={1: 'verified', 0: 'no_verified'}, inplace=True)
        
        test_val_df_replies['reply_verified'] = test_val_df_replies['reply_verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        test_val_df_replies = pd.concat([test_val_df_replies, pd.get_dummies(test_val_df_replies["reply_verified"], dtype=int)], axis=1)
        test_val_df_replies.drop(["reply_verified"], axis=1, inplace=True)
        test_val_df_replies.rename(columns={1: 'reply_verified', 0: 'reply_no_verified'}, inplace=True)
        
        test_val_df_posts = test_val_df_posts.merge(pd.concat([val,test])[['id']].reset_index(),on='id',how='left')
        test_val_df_posts.set_index('index',drop=True,inplace=True)
        
        post_features = test_val_df_posts[["followers", "favorite_count", "retweet_count", "no_verified", "verified", "first_time_diff"]]
        
        
        scaled_features = scaler_posts.transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])
        
        # Convert the scaled features back to a DataFrame
        scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])
        
        # Add the binary features back to the scaled data
        scaled_data['no_verified'] = np.array(post_features['no_verified'])
        scaled_data['verified'] = np.array(post_features['verified'])
        post_features = scaled_data
        
        post_embeddings = np.array(test_val_df_posts['embeddings_avg'].tolist())
        #post_features = scaler.fit_transform(post_features)
        x3 = np.concatenate((post_features, post_embeddings), axis=1)
        
        test_val_reply_features =  test_val_df_replies[["reply_followers", "reply_no_verified","reply_verified","time_diff"]]
        test_val_reply_features[['reply_followers','time_diff']] = scaler_replies.transform(test_val_reply_features[['reply_followers','time_diff']])
        
        test_val_reply_embeddings = np.array(test_val_df_replies['reply_embeddings_avg'].tolist())
        x4 = np.concatenate((test_val_reply_features, test_val_reply_embeddings), axis=1)
        
        
        # Mapping post ids
        post_map = {value: i for i, value in enumerate(pd.concat([train[['id']],test_val_df_posts[['id']]])['id'].unique())}
        df_replies_edges = pd.concat([self.df_replies[self.df_replies.id.isin(np.array(train.id))][["id", "reply_user_id"]],\
                                test_val_df_replies[["id", "reply_user_id"]]])
        
        df_replies_edges["id"] = df_replies_edges['id'].map(post_map).astype(int)
        
        # Mapping reply user ids
        reply_user_map = {value: i for i, value in enumerate(df_replies_edges['reply_user_id'].unique())}
        df_replies_edges["reply_user_id"] = df_replies_edges["reply_user_id"].map(reply_user_map)

        return train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4


    def create_heterodata(self,train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4):

        y = pd.concat([train['rumour'],test_val_df_posts['rumour']]).to_numpy()
        edge_index = df_replies_edges.values.transpose()
        x = np.concatenate((x1,x3))
        x_reply = np.concatenate((x2,x4))
            
        num_rows = x.shape[0]
        train_mask = np.zeros(num_rows, dtype=bool)
        val_mask = np.zeros(num_rows, dtype=bool)
        test_mask = np.zeros(num_rows, dtype=bool)
        train_mask[:-x3.shape[0]]=True
        val_mask[-x3.shape[0]:-int(x3.shape[0]/2)]=True
        test_mask[-int(x3.shape[0]/2):]=True
            
        data = HeteroData()
        data['id'].x = torch.tensor(x, dtype=torch.float32)
        data['id'].y =  torch.from_numpy(y)
        data['id'].train_mask = torch.tensor(train_mask)
        data['id'].val_mask = torch.tensor(val_mask) 
        data['id'].test_mask = torch.tensor(test_mask)
        data['reply_user_id'].x = torch.tensor(x_reply, dtype=torch.float32)
        data['id', 'retweet', 'reply_user_id'].edge_index = torch.from_numpy(edge_index.reshape(2,len(x_reply)))
        data = T.ToUndirected()(data)
        
        return data

    def process(self):
            
        self.load_data()
        train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4 = self.process_data()
        return self.create_heterodata(train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4)

In [4]:

class HeteroDataProcessorFilterNodeonTestV2:
    #Different time cuts 
    def __init__(self, file_path_replies, file_path_posts, time_cut_replies=15,time_cut_posts=15):
        self.file_path_replies = file_path_replies
        self.file_path_posts = file_path_posts
        self.time_cut_replies = time_cut_replies
        self.time_cut_posts = time_cut_posts
        self.df_replies = None
        self.df_posts = None

    def load_data(self):
        self.df_replies = pd.read_pickle(self.file_path_replies)
        self.df_posts = pd.read_pickle(self.file_path_posts)

    def process_data(self):
        # Define post and reply features
        post_features = ['followers', 'favorite_count', 'retweet_count', 'verified', 'rumour', 'id', 'embeddings_avg']
        reply_features = ['reply_followers', 'reply_user_id', 'reply_verified', 'time_diff', 'reply_embeddings_avg', 'id']

        # Filter and group replies
        self.df_replies['min_since_fst_post'] = round(
            (self.df_replies['time'] - self.df_replies['time'].min()).dt.total_seconds() / 60, 2)
        grouped_replies = self.df_replies.groupby(['id']).agg(
            replies=('time_diff', 'count'),
            first_time_diff=('time_diff', 'first')
        ).reset_index()

        # Merge posts and replies
        self.df_posts = self.df_posts[post_features].merge(grouped_replies, on="id", how="inner")
        self.df_posts['replies'] = self.df_posts['replies'].fillna(0)
        self.df_posts['first_time_diff'] = self.df_posts['first_time_diff'].fillna(0)

        # One-hot encoding for verified columns
        self.df_posts['verified'] = self.df_posts['verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        self.df_posts = pd.concat([self.df_posts, pd.get_dummies(self.df_posts["verified"], dtype=int)], axis=1)
        self.df_posts.drop(["verified"], axis=1, inplace=True)
        self.df_posts.rename(columns={1: 'verified', 0: 'no_verified'}, inplace=True)

        self.df_replies['reply_verified'] = self.df_replies['reply_verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        self.df_replies = pd.concat([self.df_replies, pd.get_dummies(self.df_replies["reply_verified"], dtype=int)], axis=1)
        self.df_replies.drop(["reply_verified"], axis=1, inplace=True)
        self.df_replies.rename(columns={1: 'reply_verified', 0: 'reply_no_verified'}, inplace=True)

        # Train/test split
        train, not_train = train_test_split(self.df_posts, test_size=0.3, random_state=42, stratify=self.df_posts['rumour'])
        val, test = train_test_split(not_train, test_size=0.5, random_state=42, stratify=not_train['rumour'])

        # Post features processing
        post_features = train[["followers", "favorite_count", "retweet_count", "no_verified", "verified", "first_time_diff"]]
        scaler_posts = RobustScaler()
        scaled_features =scaler_posts.fit_transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])
        scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])
        scaled_data['no_verified'] = np.array(train['no_verified'])
        scaled_data['verified'] = np.array(train['verified'])
        post_features = scaled_data
        post_embeddings = np.array(train['embeddings_avg'].tolist())
        x1 = np.concatenate((post_features, post_embeddings), axis=1)

        # Reply features processing
        scaler_replies = RobustScaler()
        reply_features = self.df_replies[self.df_replies.id.isin(np.array(train.id))][["reply_followers", "reply_no_verified", "reply_verified", "time_diff"]]
        reply_features[['reply_followers', 'time_diff']] = scaler_replies.fit_transform(reply_features[['reply_followers', 'time_diff']])
        reply_embeddings = np.array(self.df_replies[self.df_replies.id.isin(np.array(train.id))]['reply_embeddings_avg'].tolist())
        x2 = np.concatenate((reply_features, reply_embeddings), axis=1)

        # Test/validation data preparation
        test_val_df_replies = pd.read_pickle(self.file_path_replies)
        test_val_df_posts = pd.read_pickle(self.file_path_posts)
        test_val_df_posts = test_val_df_posts[~test_val_df_posts.id.isin(train.id)]
        test_val_df_replies = test_val_df_replies[~test_val_df_replies.id.isin(train.id)]

        post_features = ['followers', 'favorite_count', 'retweet_count', 'verified', 'rumour', 'id', 'embeddings_avg']
        test_val_reply_features = ['reply_followers', 'reply_user_id', 'reply_verified', 'time_diff', 'reply_embeddings_avg', 'id']
        
        test_val_df_replies['min_since_fst_post'] = round((test_val_df_replies['time'] - test_val_df_replies['time'].min())\
        .dt.total_seconds() / 60,2)
        
        test_val_df_replies = test_val_df_replies[test_val_reply_features][(test_val_df_replies.time_diff <= self.time_cut_replies)&\
           (test_val_df_replies.min_since_fst_post <= self.time_cut_posts)]
        grouped_replies = test_val_df_replies.groupby(['id']).agg(
            replies=('time_diff', 'count'),
            first_time_diff=('time_diff', 'first')
        ).reset_index()
        
        test_val_df_posts = test_val_df_posts[post_features].merge(grouped_replies, on="id", how="inner")
        test_val_df_posts['replies'] = test_val_df_posts['replies'].fillna(0)
        test_val_df_posts['first_time_diff'] = test_val_df_posts['first_time_diff'].fillna(0)
        
        # One-hot encoding for verified columns
        test_val_df_posts['verified'] = test_val_df_posts['verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        test_val_df_posts = pd.concat([test_val_df_posts, pd.get_dummies(test_val_df_posts["verified"], dtype=int)], axis=1)
        test_val_df_posts.drop(["verified"], axis=1, inplace=True)
        test_val_df_posts.rename(columns={1: 'verified', 0: 'no_verified'}, inplace=True)
        
        test_val_df_replies['reply_verified'] = test_val_df_replies['reply_verified'].astype(str).replace({'True': '1', 'False': '0'}).astype(int)
        test_val_df_replies = pd.concat([test_val_df_replies, pd.get_dummies(test_val_df_replies["reply_verified"], dtype=int)], axis=1)
        test_val_df_replies.drop(["reply_verified"], axis=1, inplace=True)
        test_val_df_replies.rename(columns={1: 'reply_verified', 0: 'reply_no_verified'}, inplace=True)
        
        test_val_df_posts = test_val_df_posts.merge(pd.concat([val,test])[['id']].reset_index(),on='id',how='left')
        test_val_df_posts.set_index('index',drop=True,inplace=True)
        
        post_features = test_val_df_posts[["followers", "favorite_count", "retweet_count", "no_verified", "verified", "first_time_diff"]]
        
        
        scaled_features = scaler_posts.transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])
        
        # Convert the scaled features back to a DataFrame
        scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])
        
        # Add the binary features back to the scaled data
        scaled_data['no_verified'] = np.array(post_features['no_verified'])
        scaled_data['verified'] = np.array(post_features['verified'])
        post_features = scaled_data
        
        post_embeddings = np.array(test_val_df_posts['embeddings_avg'].tolist())
        #post_features = scaler.fit_transform(post_features)
        x3 = np.concatenate((post_features, post_embeddings), axis=1)
        
        test_val_reply_features =  test_val_df_replies[["reply_followers", "reply_no_verified","reply_verified","time_diff"]]
        test_val_reply_features[['reply_followers','time_diff']] = scaler_replies.transform(test_val_reply_features[['reply_followers','time_diff']])
        
        test_val_reply_embeddings = np.array(test_val_df_replies['reply_embeddings_avg'].tolist())
        x4 = np.concatenate((test_val_reply_features, test_val_reply_embeddings), axis=1)
        
        
        # Mapping post ids
        post_map = {value: i for i, value in enumerate(pd.concat([train[['id']],test_val_df_posts[['id']]])['id'].unique())}
        df_replies_edges = pd.concat([self.df_replies[self.df_replies.id.isin(np.array(train.id))][["id", "reply_user_id"]],\
                                test_val_df_replies[["id", "reply_user_id"]]])
        
        df_replies_edges["id"] = df_replies_edges['id'].map(post_map).astype(int)
        
        # Mapping reply user ids
        reply_user_map = {value: i for i, value in enumerate(df_replies_edges['reply_user_id'].unique())}
        df_replies_edges["reply_user_id"] = df_replies_edges["reply_user_id"].map(reply_user_map)

        return train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4


    def create_heterodata(self,train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4):

        y = pd.concat([train['rumour'],test_val_df_posts['rumour']]).to_numpy()
        edge_index = df_replies_edges.values.transpose()
        x = np.concatenate((x1,x3))
        x_reply = np.concatenate((x2,x4))
            
        num_rows = x.shape[0]
        train_mask = np.zeros(num_rows, dtype=bool)
        val_mask = np.zeros(num_rows, dtype=bool)
        test_mask = np.zeros(num_rows, dtype=bool)
        train_mask[:-x3.shape[0]]=True
        val_mask[-x3.shape[0]:-int(x3.shape[0]/2)]=True
        test_mask[-int(x3.shape[0]/2):]=True
            
        data = HeteroData()
        data['id'].x = torch.tensor(x, dtype=torch.float32)
        data['id'].y =  torch.from_numpy(y)
        data['id'].train_mask = torch.tensor(train_mask)
        data['id'].val_mask = torch.tensor(val_mask) 
        data['id'].test_mask = torch.tensor(test_mask)
        data['reply_user_id'].x = torch.tensor(x_reply, dtype=torch.float32)
        data['id', 'retweet', 'reply_user_id'].edge_index = torch.from_numpy(edge_index.reshape(2,len(x_reply)))
        data = T.ToUndirected()(data)
        
        return data

    def process(self):
            
        self.load_data()
        train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4 = self.process_data()
        return self.create_heterodata(train,test_val_df_posts,df_replies_edges,x1,x2,x3,x4)

In [10]:

# Usage
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"
time_cut =12

processor = HeteroDataProcessorFilterNodeonTestV2(file_path_replies, file_path_posts, time_cut_replies=10,time_cut_posts=50)
data = processor.process()


/tmp/ipykernel_5639/450977190.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_val_reply_features[['reply_followers','time_diff']] = scaler_replies.transform(test_val_reply_features[['reply_followers','time_diff']])


In [11]:
data

HeteroData(
  id={
    x=[1430, 106],
    y=[1430],
    train_mask=[1430],
    val_mask=[1430],
    test_mask=[1430],
  },
  reply_user_id={ x=[13286, 104] },
  (id, retweet, reply_user_id)={ edge_index=[2, 13286] },
  (reply_user_id, rev_retweet, id)={ edge_index=[2, 13286] }
)

In [9]:
class LoadRumoursDatasetFilterNodeV2:
    def __init__(self, file_path_replies, file_path_posts, time_cut_replies=15,time_cut_posts=15):
        self.file_path_replies = file_path_replies
        self.file_path_posts = file_path_posts
        self.time_cut_replies = time_cut_replies
        self.time_cut_posts = time_cut_posts
        self.df_replies = None
        self.df_posts = None
        self.df_final = None

    def load_data(self):
        self.df_replies = pd.read_pickle(self.file_path_replies)
        self.df_posts = pd.read_pickle(self.file_path_posts)

    def process_data(self):
        post_features = ['followers','favorite_count','retweet_count','verified','rumour','id','embeddings_avg']
        
        
        self.df_replies['min_since_fst_post'] = round((self.df_replies['time'] - self.df_replies['time'].min())\
                        .dt.total_seconds() / 60,2)
        
        reply_features = ['reply_followers','reply_user_id','reply_verified','time_diff','reply_embeddings_avg',\
                          'min_since_fst_post','id','time']

        filtered_replies = self.df_replies[reply_features][(self.df_replies.time_diff <= self.time_cut_replies)&\
                                                           (self.df_replies.min_since_fst_post <= self.time_cut_posts)]
        
        grouped_replies = filtered_replies.groupby(['id']).agg(
            replies=('time_diff', 'count'),
            first_time_diff=('time_diff', 'first')
        ).reset_index()

        self.df_posts = self.df_posts[post_features]
        self.df_final = self.df_posts.merge(grouped_replies, on="id", how="inner")
        self.df_final['replies'] = self.df_final['replies'].fillna(0)
        self.df_final['first_time_diff'] = self.df_final['first_time_diff'].fillna(0)
        #self.df_final = self.df_final.drop(columns=['id'])

        
        # Initialize the Robust Scaler
        scaler = RobustScaler()
        
        # Assuming data is a DataFrame containing your dataset
        scaled_features = ['followers', 'favorite_count', 'retweet_count', 'first_time_diff']
        # Convert the scaled features back to a DataFrame
        scaled_data = pd.DataFrame(scaler.fit_transform(self.df_final [scaled_features]),columns=scaled_features)    
        
        self.df_final [scaled_features] = scaled_data


        # One-hot encoding
        self.df_final ['verified'] = self.df_final ['verified'].astype('str').str.\
                     replace(' ', '').replace('True', '1').replace('False', '0')\
                     .astype('int64')
        
        self.df_final  = pd.concat([self.df_final , pd.get_dummies(\
                                  self.df_final ["verified"],dtype=int)], axis=1, join='inner')
        self.df_final .drop(["verified"], axis=1, inplace=True)
        self.df_final .rename(columns={1:'verified',0:'no_verified'},inplace=True)

    def get_final_dataframe(self):
        return self.df_final

In [10]:
#
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"
time_cut = 1e10

processor = LoadRumoursDatasetFilterNodeV2(file_path_replies, file_path_posts,  time_cut_replies=10,time_cut_posts=50)
processor.load_data()
processor.process_data()
df_final = processor.get_final_dataframe()


In [11]:
df_final

,followers,favorite_count,retweet_count,rumour,id,embeddings_avg,replies,first_time_diff,no_verified,verified
0,-0.007584,0.050633,1.648649,1,552783667052167168,"[-0.12335950043052435, -0.055849663292368255, ...",1,2.189091,0,1
1,0.287235,-0.531646,-0.291892,1,552783745565347840,"[-0.1364929385483265, -0.07159566258390744, -0...",2,1.680000,0,1
2,0.265037,-0.531646,-0.410811,1,552784168849907712,"[-0.045377860377941816, -0.20127306692302227, ...",2,-0.101818,0,1
3,0.911401,-0.101266,1.248649,1,552784526955806720,"[-0.03706469060853124, -0.1309182441327721, -0...",4,-0.472727,0,1
4,1.213457,0.303797,1.778378,1,552784882687299584,"[-0.003344586119055748, 0.11169316650678714, 0...",8,0.000000,0,1
...,...,...,...,...,...,...,...,...,...,...
74,-0.098331,4.329114,1.205405,0,552793409438904321,"[-0.1536131378961727, 0.25495246875410277, 0.0...",9,-0.298182,1,0
75,9.406947,2.835443,2.783784,0,552794286849536001,"[-0.21957818283276123, -0.17643718150528995, 0...",16,-0.378182,0,1
76,1.212691,0.151899,0.356757,0,552794514424102913,"[0.21425410360097885, -0.1142502959817648, 0.1...",5,-0.341818,0,1
77,-0.096839,1.772152,0.681081,0,552794995942772736,"[0.10138966764012973, -0.06978333741426468, 0....",1,1.818182,1,0
